## Original script

In [ ]:
#imports
import pandas as pd
import os 
from pathlib import Path
import copy

import numpy as np
import scipy as stats
import random

import seaborn as sns
import matplotlib.pyplot as plt

import scipy as sp
from scipy.stats import chi2
from sklearn.covariance import MinCovDet

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression


In [ ]:

'''
Step 1: Filter out the non-protein coding genes from GTEX

Remove genes from the data which do not encode a protein and to log transform the gene expression matrix.

Source:  https://www.genenames.org/download/statistics-and-files/ retrieved January 19 2022 </p>
Selected 19208 protein coding genes as txt file </p>
Input: protein coding gene list with current list of know protein coding genes
       GTEX matrix contain at total of 56200 genes and 17382 samples 

Output: 
A matrix containing on the ENSMBL description and only the genes that were in the intersection of the protein coding genes and GTEx matrix. 
A dictionary from ENSMBL to description.
'''

def filter_non_protein_coding(HGNC_list, GTEx_tpm):
    
    PCG_list.rename(columns={'symbol':'PCG Symbols'},inplace=True)
    PCG_list = PCG_list['PCG Symbols'].tolist()

    matrix_pcg_df = GTEx_tpm[GTEx_tpm['Description'].isin(PCG_list)]
    pcg_dictionary = matrix_pcg_df[['Name','Description']]
    pcg_dictionary = pcg_dictionary.set_index('Name')
    
    matrix_pcg_df =matrix_pcg_df.drop(['Description'], axis=1)
    matrix_pcg_df.set_index('Name',inplace=True)
    
    log_transform(matrix_pcg_df)
    
    return pcg_dictionary,matrix_pcg_df


def log_transform(matrix_pcg_df):
    matrix_pcg_df = matrix_pcg_df +1
    matrix_pcg_df = np.log2(matrix_pcg_df)
    return matrix_pcg_df


In [ ]:

'''
Step 2: Sample attributes
1. Filter out death types for both samples and attributes 
2. Filter out low RIN

Input: Sample annotations (attributes csv file),
       Subject phenotypes (pheno csv file),
       GTEX matrix
'''

def setup_attributes (pheno_df, attributes_df, matrix_pcg_df):
    # Filter death types
    pheno_a = pheno[(pheno['DTHHRDY'] > 0) & (pheno['DTHHRDY'] < 3)]
    important_attributes = attributes[['SMRIN','SMTSISCH','SMTSD','SMGEBTCH']]
    important_attributes = important_attributes.assign(SUBJID = '')
    sampids = important_attributes.index.tolist()
    for i in range(important_attributes.shape[0]):
        x = sampids[i].split('-')
        xx = ''
        xx = xx+x[0]+'-'+x[1]
        important_attributes.iloc[i,4] = xx
    important_attributes.dropna(inplace = True)
    
    # Filter RIN
    important_attributes = important_attributes.drop(important_attributes[important_attributes['SMRIN'] <5.7].index, axis = 0)

    new_df  =pd.merge(important_attributes.reset_index(), pheno_a, on=['SUBJID'], how='inner')
    new_df =new_df.set_index('SAMPID')
    pheno_aa = new_df.dropna(axis=0,how='any')
    pheno_aa['AGE'] = pheno_aa['AGE'].str.split('-', 1).str[0].astype(int)
    
    aa_samples = pheno_aa.index.tolist()
    matrix_pcg_aa = matrix_pcg.loc[:, matrix_pcg.columns.isin(aa_samples)]
    aa_matrix_samples = matrix_pcg_aa.columns.to_list()
    sample_attributes = new_df.loc[new_df.index.isin(aa_matrix_samples),:]
    return matrix_pcg_aa, sample_attributes


In [ ]:

'''
Step 3: Create data for tissues
'''
def create_data_for_tissue (sample_attributes, matrix_pcg_aa, number_of_tissues = 13, length_threshold = 100):

    samples = sample_attributes
    sample_tissues = samples['SMTSD'].value_counts()

    tissues = sample_tissues[sample_tissues>100].index.to_list()
    pcg = matrix_pcg_aa

    for i in range(number_of_tissues):

        current_tissue = tissues[i]
        current_tissue = current_tissue[:length_threshold]

        tissue_type = current_tissue[:length_threshold]

        tissue_samples = samples[samples['SMTSD'].str.startswith(tissue_type)]
        tissue_ids = tissue_samples.index.tolist()

        print(pheno.shape)
        matrix = pcg[tissue_ids]
        
        tissue_samples.to_csv(f'{current_tissue}/tissue_sample_DT1DT2.csv')
        matrix.to_csv(f'{current_tissue}/tissue_matrix_DT1DT2.csv')


In [ ]:
  
'''
Step 4: Process tissue data

1. Filter genes with low expression and low veriabilty 
2. Remove outliers
3. Quantile normalization
4. Make modifications to phenotype attributes
'''

def create_exclusion_list(genes, values, limit_val):
    exclude_genes_list = []
    for i in range(len(genes)):
        if (values[i] < limit_val):
            exclude_genes_list.append(genes[i])
    return exclude_genes_list 


# filter Genes with zero variance  
def filter_genes (tissue_matrix, sample_attributes):
    sample_attributes['AGE'] = sample_attributes['AGE'].str.split('-', 1).str[0].astype(int)

    print(str(len(tissue_matrix[(tissue_matrix.T <np.log2(0.1+1) ).sum()>0.2*tissue_matrix.shape[1]])) + " genes filtered out")
    df1 = tissue_matrix[(tissue_matrix.T <np.log2(0.1+1) ).sum()<0.2*tissue_matrix.shape[1]] 
    df1 = df1.T
    variability_df = df1.var()
    print(variability_df)
    gene_names = variability_df.index.tolist()
    gene_vars = variability_df.tolist()

    excluded_list = create_exclusion_list(gene_names,gene_vars,variability_threshold)
    print('length of excluded list:' + str(len(excluded_list)))

    df1 = df1.drop(excluded_list, axis=1)
    sample1 = sample_attributes.copy(deep=True)
    return df1, sample1
  

#remove outlier samples using mahanalobis on specific pca
def sample_outliers_df(matrix_val, sample_val):
    print('sample outliers')
    print(matrix_val.shape) # rows are genes col are sampels
    display(matrix_val.head())
    display(sample_val.head())
    print('isna')
    print(matrix_val.isna().sum())
    matrix_val.isna().sum()
    matrix_val.fillna(value=0.0000001,inplace=True)
    print(matrix_val.isna().sum())
    
    sample_cum_pca = []
    gene_cum_pca = []
    current_sum = 0
    print("removing outliers")
    print(matrix_val.shape)
    #pca_gene = PCA(n_components=gene_pca_components)
    pca_gene = TruncatedSVD(n_components=20, random_state=1001)
    pca_gene.fit(matrix_val)
    components =  pca_gene.components_
    components = components.T
    print(type(components))
    print('components shape')
    print(components.shape)
    comp_df = pd.DataFrame(data=components)
    
    samples_before_outlier_removal = matrix_val.columns.tolist()
    print(samples_before_outlier_removal)
    samples_to_remove = []
    
     
    # Covariance matrix
    covariance  = np.cov(components , rowvar=False)
    # Covariance matrix power of -1
    covariance_pm1 = np.linalg.matrix_power(covariance, -1)
    # Center point
    centerpoint = np.mean(components , axis=0)
    
    # Distances between center point and 
    distances = []
    for i, val in enumerate(components):
        p1 = val
        
        p2 = centerpoint
       
        distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
        distances.append(distance)
    distances = np.array(distances)

    
    # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
    cutoff = chi2.ppf(0.99, components.shape[1]*2)
    print('distances')
    print(distances)
    print('cutoff')
    print(cutoff)
    display(matrix_val.head())
    print(matrix_val.shape)
    display(sample_val.head())
    print(sample_val.shape)

    # Index of outliers
    outlierIndexes = np.where(distances > cutoff )
    outlierIndexes = list(outlierIndexes[0])

    print('--- Index of Outliers ----')
    print(outlierIndexes)
    print(type(outlierIndexes))
    print(len(outlierIndexes))
    for q2 in range(len(outlierIndexes)):
        print(outlierIndexes[q2])
        samples_to_remove.append(samples_before_outlier_removal[q2])
    print(samples_to_remove)
    
    df = pd.DataFrame(data=components)
    display(df.head())
    
    print('dropping outliers')
    print('matrix shape before drop')
    print(matrix_val.shape)
    matrix_val = matrix_val.drop(labels = samples_to_remove, axis=1)
    print('matrix shape after drop')
    print(matrix_val.shape)
    print('sample shape before drop')
    print(sample_val.shape)
    sample_val = sample_val.drop(labels = samples_to_remove, axis=0)
    print('sample shape after drop')
    print(sample_val.shape)

    return matrix_val, sample_val


def quantile_normalize(matrix_val):
    df = matrix_val
    df_image1 = df.iloc[:,0:3]
    print('df image before quantile normalization: '+str(df_image1.shape))
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    ax = sns.boxplot(data=df_image1, linewidth=2.5).set(title='Before quantile')
    plt.savefig('before_quantile.png')
    plt.show()
    print(matrix_val.shape)
    display(df_image1.head(2))

    df_sorted = pd.DataFrame(np.sort(df.values,
                                     axis=0), 
                             index=df.index, 
                             columns=df.columns)
    df_mean = df_sorted.mean(axis=1)
    df_mean.index = np.arange(1, len(df_mean) + 1)
    df_qn =df.rank(method="min").stack().astype(int).map(df_mean).unstack()
    
    
    df_image2 = df_qn.iloc[:,0:3]
    print('df image after quantile normalization: '+str(df_image2.shape))
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    ax = sns.boxplot(data=df_image2, linewidth=2.5).set(title='After quantile')
    plt.savefig('after_quantile.png')
    plt.show()
    display(df_image2.head(2))
    
    return df_qn



def process_tissue (tissue_matrix , sample_attributes, current_tissue):

    # 1.filter genes with low expression and low veriabilty
    print("---1.filtering---")
    ###remove genes that have value less than 0.1 transcripts per million (TPM) in more than 80% of the samples
    df1, sample1 = filter_genes(tissue_matrix, sample_attributes)


    # 2. remove outliers
    print("---2.remove outliers---")
    df2, sample2 = sample_outliers_df(df1.T,sample1)

    # 3. quantile normalization
    print("---3.quantile normalization---")

    df3 = quantile_normalize(df2)

    # 4. make modifications to phenotype attributes

    sample3 = sample2.copy(deep=True)
    agegroup = sample3['AGE'].tolist()
    sample3['AGE'] = [elem[:2] for elem in agegroup]
    SMGEBTCH= sample3['SMGEBTCH'].value_counts(normalize=False, sort=True)
    SMGEBTCH_names = SMGEBTCH.index.tolist()
    SMGEBTCH_value =  SMGEBTCH.tolist()
    SMGEBTCH_new_list  = []
    for i in range (len(SMGEBTCH_names)):
        if SMGEBTCH_value[i] > 1:
            SMGEBTCH_new_list.append(SMGEBTCH_names[i])

    batches = []
    for i in range(sample3.shape[0]):

        if sample3['SMGEBTCH'].iloc[i] in SMGEBTCH_new_list:
            batches.append(sample3['SMGEBTCH'].iloc[i])
        else:
            batches.append('ASINGLETON_SMGEBTCH')

    sample3['SMGEBTCH'] = batches

    SMGEBTCH = pd.get_dummies(sample4['SMGEBTCH'],drop_first=True)
    sample4.drop(['SMTSD','SMGEBTCH','SUBJID'], axis=1, inplace=True)

    new_sample4 = [sample3, SMGEBTCH]
    sample5 = pd.concat(new_sample4 , join='inner', axis=1)
    
          
    df3.to_csv(f'{current_tissue}/tissue_matrix_preReg.csv')
    sample5.to_csv(f'{current_tissue}/tissue_sample_preReg.csv')
    return df3, sample5


In [ ]:

'''
Step 5: Confouding factors
'''

def regress_confounding (matrix , sample, current_tissue):
    matrix = matrix.T # sampels X genes 
    sk_resid_mat=matrix.copy(deep=True) # sampels X genes 
    for col in sk_resid_mat.columns: # loop on genes
        sk_resid_mat[col].values[:] = 0

    age =  sample['AGE']
    death_type =  sample['DTHHRDY']
    
    x = sample.drop(['AGE', 'SMRIN', 'DTHHRDY'], axis =1)
    
    for i in range(matrix.shape[1]): # loop on cols = genes
        reg = LinearRegression()
        y = matrix.iloc[:,i] # all sampels, one gene
        reg.fit(x,y)
        prediction = reg.predict(x)
        coefs = reg.coef_.tolist()
        sk_resid_mat.iloc[:,i] = y - prediction # replace gene i with residual
              
    feature_names = sample.columns.tolist()

    sk_resid_mat_norm = quantile_normalize(sk_resid_mat.T)
    sk_resid_mat_norm.to_csv(f'{current_tissue}/sk_residual_matrix_w_const.csv')
    return sk_resid_mat_norm


In [ ]:
  
'''
Step 6: split age groups
'''

def split_age_group(matrix, sample_processed, current_tissue)

    matrix = matrix.T
    pool_of_samples = samples.index.tolist() 
    
    young_sample =  sample[sample['AGE'] < 60]
    old_sample =    sample[sample['AGE'] >= 60]  
    
    young_list =  young_sample.index.tolist()
    old_list =  old_sample.index.tolist()
    
    young_matrix = matrix[matrix.columns.intersection(young_list)]
    young_matrix = young_matrix.T
   
    old_matrix = matrix[matrix.columns.intersection(old_list)]
    old_matrix =  old_matrix.T
    
    young_sample.to_csv(f'{current_tissue}/young_sample.csv')
    young_matrix.to_csv(f'{current_tissue}/young_matrix.csv')
    old_sample.to_csv(f'{current_tissue}/old_sample.csv')
    old_matrix.to_csv(f'{current_tissue}/old_matrix.csv')
    
    return young_sample, young_matrix, old_sample, old_matrix
 



# Reviewing the old scripts

## GTEx_dataPrep.py
a. This script starts with the main function that loads the GTEx data, HGNC gene names, phenotype data, and attribute data.
b. Then it initializes some constant variables and tissues target list.

 ```python
if __name__ == '__main__':
    
    GTEx_tpm = pd.read_csv('GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct', sep='\t',skiprows=2)
    HGNC_list = pd.read_csv('ProteinCodingGenesRetreived01192022.csv', usecols=[0,1,2,3,4,5,6])
    pheno_df = pd.read_csv('GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.csv')
    attributes_df = pd.read_csv('GTEx_Analysis_v8_Annotations_SampleAttributesDS.csv',index_col=0)

    number_of_tissues = 13
    length_threshold = 100
    variability_threshold = 0.02
    sample_pca_components = 3
    gene_pca_components = 50

    tissues = ['Muscle - Skeletal',
     'Whole Blood',
     'Skin - Sun Exposed (Lower leg)',
     'Skin - Not Sun Exposed (Suprapubic)',
     'Adipose - Subcutaneous',
     'Thyroid',
     'Artery - Tibial',
     'Nerve - Tibial',
     'Lung',
     'Brain - Cerebellum',
     'Heart - Atrial Appendage',
     'Brain - Cortex',
     'Adipose - Visceral (Omentum)']
```

c. The script then calls the `filter_non_protein_coding` function with the HGNC list and the GTEx TPM data to filter out non-protein coding genes.

```python
    pcg_dictionary,matrix_pcg_df = filter_non_protein_coding(HGNC_list, GTEx_tpm)
```

d. The `filter_non_protein_coding` function filter the non-protein coding genes from the GTEx TPM data and returns a dictionary of protein coding genes and a DataFrame of the filtered matrix. But this function also includes a `log_transform` function that applies a log transformation to the matrix before returning it. Thats a bug in the original script, as the log transformation is happening again in the `main` function after the data is filtered.

e. Then the script calls the `setup_attributes` function with the phenotype data, attributes data, and the filtered matrix to:
    1. filter the death types (only 1,2)
    2. transform the subject IDs to short IDs
    3. filter the low quality samples (RIN < 5.7)
    4. convert the age to numeric (60-79 to 60)

f. Then the script calls the `create_data_for_tissue` function to save the data for each tissue after the processing steps.

g. Then the script iterates over the list of tissues and calls the `process_tissue` function for each tissue to:
    1. filter genes by variability threshold (and convert age to numeric again)
    2. remove outliers from the matrix by Hotellinger's T2 test:

```python
    def sample_outliers_df(matrix_val, sample_val):
    matrix_val.isna().sum()
    matrix_val.fillna(value=0.0000001,inplace=True)
    
    sample_cum_pca = []
    gene_cum_pca = []
    current_sum = 0
    #pca_gene = PCA(n_components=gene_pca_components)
    pca_gene = TruncatedSVD(n_components=20, random_state=1001)
    pca_gene.fit(matrix_val)
    components =  pca_gene.components_
    components = components.T
    comp_df = pd.DataFrame(data=components)
    
    samples_before_outlier_removal = matrix_val.columns.tolist()
    samples_to_remove = []
    
     
    # Covariance matrix
    covariance  = np.cov(components , rowvar=False)
    # Covariance matrix power of -1
    covariance_pm1 = np.linalg.matrix_power(covariance, -1)
    # Center point
    centerpoint = np.mean(components , axis=0)
    
    # Distances between center point and 
    distances = []
    for i, val in enumerate(components):
        p1 = val
        
        p2 = centerpoint
       
        distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
        distances.append(distance)
    distances = np.array(distances)

    
    # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
    cutoff = chi2.ppf(0.99, components.shape[1]*2)

    # Index of outliers
    outlierIndexes = np.where(distances > cutoff )
    outlierIndexes = list(outlierIndexes[0])

    for q2 in range(len(outlierIndexes)):
        samples_to_remove.append(samples_before_outlier_removal[q2])
    
    df = pd.DataFrame(data=components)
    
    matrix_val = matrix_val.drop(labels = samples_to_remove, axis=1)
    sample_val = sample_val.drop(labels = samples_to_remove, axis=0)

    return matrix_val, sample_val
```
* NOTE the script is iterating over an index from zero to the length of the outlier indexes, but it should iterate over the `outlierIndexes` list directly.

$$d^2(x_i) = (x_i - \mu)^T \Sigma^{-1} (x_i - \mu)$$

Where:

- $x_i$ is sample $i$ in the PCA-reduced space
- $\mu$ is the mean vector across all samples
- $\Sigma^{-1}$ is the inverse of the covariance matrix

    3. apply quantile normalization to the matrix
    4. prepare the sample gene expression batch data for the dummy transformation (for the confounding variables)
    5. save the processed data to a CSV file

h. The script calls the `regress_confounding` function with the processed matrix and the sample data to regress out the confounding variables.

```python
def regress_confounding (matrix , sample, current_tissue):
    matrix = matrix.T # sampels X genes 
    sk_resid_mat=matrix.copy(deep=True) # sampels X genes 
    for col in sk_resid_mat.columns: # loop on genes
        sk_resid_mat[col].values[:] = 0

    age =  sample['AGE']
    death_type =  sample['DTHHRDY']
    
    x = sample.drop(['AGE', 'SMRIN', 'DTHHRDY'], axis =1)
    
    for i in range(matrix.shape[1]): # loop on cols = genes
        reg = LinearRegression()
        y = matrix.iloc[:,i] # all sampels, one gene
        reg.fit(x,y)
        prediction = reg.predict(x)
        coefs = reg.coef_.tolist()
        sk_resid_mat.iloc[:,i] = y - prediction # replace gene i with residual
              
    feature_names = sample.columns.tolist()

    sk_resid_mat_norm = quantile_normalize(sk_resid_mat.T)
    sk_resid_mat_norm.to_csv(f'{current_tissue}/sk_residual_matrix_w_const.csv')
    return sk_resid_mat_norm
```
* NOTE the script is not dropping all the non-numeric columns from the sample data before the regression, which can lead to errors in the regression model. Also, the script is not converting the boolean columns to integers (0,1) before the regression, which can lead to errors in the regression model too.
Also, the script is calling the `quantile_normalize` function on the residual matrix, I wonder if this is a bug or not, as the residual matrix should already be normalized.
Finally, the script run the regression for each gene separately, which is not efficient. It would be better to run the regression in vectorized form for all genes at once.

i. The script then calls the `split_age_group` function to split the matrix into age groups (around 60) and save the data for each age group.



In [ ]:
# Original script

